# climpy — AMO, Tripole, Corelații, Regresii, CCA

**Flux complet:**
1. 📦 Instalare climpy + dependențe
2. 🌊 Descărcare date (SST, SAT Berkeley Earth, SLP, Precip GPCP global)
3. 🔬 Preprocessing + EOF → AMO + Tripole
4. 🗺 Figuri EOF (Nature-style)
5. 📊 Corelații cu stippling FDR — AMO & Tripole × (SAT / SLP / Precip)
6. 📈 Regresii cu corecție autocorelare + stippling — AMO & Tripole × (SAT / SLP / Precip)
7. 🔗 CCA (SST × SLP) — moduri coupled AMOC-NAO

---

**Date folosite și tipul lor:**

| Dataset | Sursă | Tip date | Variabilă | Notă |
|---|---|---|---|---|
| SST ERSSTv5 | NOAA PSL | **raw** monthly means | `sst` | anomalii calculate de preprocess |
| SAT BEST | Berkeley Earth | **deja anomalii** (rel. 1951-1980) | `temperature` | full global, fără goluri |
| SLP NCEP-R1 | NOAA PSL | **raw** monthly means (Pa) | `slp` | anomalii calculate de preprocess |
| Precip GPCP v2.3 | NOAA PSL | **raw** monthly means (mm/day) | `precip` | global (land+ocean) |

> ℹ️ **De ce Berkeley Earth pentru SAT?**  
> GISTEMP interpolează doar la 250km în jurul stațiilor → goluri mari în Africa, Asia Centrală, Arctic.  
> Berkeley Earth folosește kriging pe toată planeta → acoperire practic completă, recomandat pentru teleconexiuni AMOC.

Rulează celulele **în ordine de sus în jos**.

## 0 — Setup Colab

**Pasul 1:** Încarcă `climpy_fixed_v3.zip` în Colab (📂 → Upload)  
**Pasul 2:** Rulează celulele de mai jos în ordine.

In [ ]:
import subprocess, sys, os

# ── Instalare pachete ────────────────────────────────────────────────────────
pkgs = ['cartopy','eofs','xarray','numpy','pandas',
        'matplotlib','scipy','netCDF4','cmocean','cftime']
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs, check=True)
print('✓ Pachete instalate')

# ── Instalare climpy din zip ─────────────────────────────────────────────────
ZIP_NAME = 'climpy_fixed_v3.zip'
if os.path.exists(ZIP_NAME):
    subprocess.run(['unzip', '-q', '-o', ZIP_NAME, '-d', '/content'], check=True)
    climpy_path = '/content/climpy_fixed'
    if climpy_path not in sys.path:
        sys.path.insert(0, climpy_path)
    print(f'✓ climpy instalat din {ZIP_NAME}')
else:
    print(f'⚠️  {ZIP_NAME} nu a fost găsit — încarcă zip-ul în Colab (📂 → Upload)')

import climpy
print(f'✓ climpy {climpy.__version__} gata')

In [ ]:
# ── Descărcare date (~10 min la prima rulare, salvate local) ─────────────────
os.makedirs('data/real', exist_ok=True)
os.makedirs('figures',   exist_ok=True)

DATASETS = {
    # SST: ERSSTv5 — ~35 MB — 1854–prezent, global, 2° — RAW monthly means
    'data/real/sst.nc': (
        'https://downloads.psl.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc',
        'SST ERSSTv5  (raw, ~35 MB)'
    ),
    # SAT: Berkeley Earth Land+Ocean — ~180 MB — 1850–prezent, 1° — DEJA ANOMALII
    'data/real/sat_best.nc': (
        'https://berkeley-earth-temperature.s3.amazonaws.com/Global/Gridded/Land_and_Ocean_LatLong1.nc',
        'SAT Berkeley Earth (deja anomalii, ~180 MB)'
    ),
    # SLP: NCEP Reanalysis 1 — ~35 MB — 1948–prezent, global, Pa — RAW monthly means
    'data/real/slp.nc': (
        'https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.mon.mean.nc',
        'SLP NCEP-R1  (raw, ~35 MB)'
    ),
    # Precip: GPCP v2.3 — ~15 MB — 1979–prezent, GLOBAL (land+ocean), mm/day — RAW
    'data/real/precip.nc': (
        'https://downloads.psl.noaa.gov/Datasets/gpcp/precip.mon.mean.nc',
        'Precip GPCP v2.3 (raw global, ~15 MB)'
    ),
}

for dest, (url, label) in DATASETS.items():
    if os.path.exists(dest):
        mb = os.path.getsize(dest) // 1024 // 1024
        print(f'  ✓ {label:45s} deja descărcat ({mb} MB)')
    else:
        print(f'  ↓ {label:45s} ...')
        ret = os.system(f'wget -q --show-progress -O {dest} "{url}"')
        if ret == 0:
            mb = os.path.getsize(dest) // 1024 // 1024
            print(f'    ✓ {mb} MB')
        else:
            print(f'    ✗ EROARE — verifică conexiunea')
print('\nGata.')

## 1 — Imports & stil

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import xarray as xr
import climpy
from climpy.analysis import correlation_pvalue, regression_pvalue
from climpy.analysis import CCA

%matplotlib inline
plt.rcParams['figure.dpi'] = 150
climpy.use_style('nature')

DATA = Path('data/real')

CMAP_SST    = cmo.balance
CMAP_SAT    = cmo.balance
CMAP_PRECIP = cmo.tarn
CMAP_SLP    = cmo.curl

# Perioadă comună (intersecție SST+BEST+SLP+GPCP — limitată de GPCP din 1979)
PERIOD = ('1979-01', '2019-12')
REF    = ('1981-01', '2010-12')   # perioadă WMO standard

print('✓ Gata')

## 2 — Preprocessing

### 2a — SST ERSSTv5 (raw → anomalii calculate)

In [ ]:
sst_r = climpy.preprocess(
    DATA / 'sst.nc',
    var            = 'sst',
    fill_value     = -9.96921e+36,
    lon_convention = '[-180,180]',
    time           = PERIOD,
    lat            = (0, 80),
    lon            = (-76, 20),
    ref_period     = REF,
    compute_annual = True,
    subtract_gm    = True,       # elimina trend GW → variabilitate internă AMO
    # already_anomalies=False implicit — ERSSTv5 sunt raw monthly means
)
annual_sst = sst_r['annual_no_gm'].dropna('year', how='all')
print(f'SST: {dict(annual_sst.sizes)},  ani {int(annual_sst.year[0])}–{int(annual_sst.year[-1])}')

### 2b — SAT Berkeley Earth (deja anomalii — fără double-anomalising)

In [ ]:
# load_berkeley_earth() gestionează automat:
#   - conversia timpului decimal (1981.125 → Feb 1981)
#   - redenumirea latitude/longitude → lat/lon
#   - subsetul spațial și temporal
# Returnează direct DataArray cu anomalii gata de folosit.

sat_da = climpy.load_berkeley_earth(
    DATA / 'sat_best.nc',
    time           = PERIOD,
    lon_convention = '[-180,180]',
)
print(f'BEST brut: {dict(sat_da.sizes)}')

# Medii sezoniere direct (deja anomalii — NU mai cheamă anomalies() !)
sat_djfm = climpy.seasonal_means(sat_da, months=[12,1,2,3]).dropna('year', how='all')
sat_jjas = climpy.seasonal_means(sat_da, months=[6,7,8,9]).dropna('year', how='all')
print(f'SAT DJFM: {dict(sat_djfm.sizes)}')
print(f'SAT JJAS: {dict(sat_jjas.sizes)}')

### 2c — SLP NCEP-R1 și Precip GPCP (raw → anomalii calculate)

In [ ]:
# SLP: raw monthly means (Pa) → preprocess calculează anomalii față de REF
slp_r = climpy.preprocess(
    DATA / 'slp.nc', var='slp',
    lon_convention='[-180,180]',
    time=PERIOD, ref_period=REF,
    compute_annual=False, subtract_gm=False,
    # already_anomalies=False implicit
)
slp_djfm = climpy.seasonal_means(slp_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
slp_djfm = slp_djfm / 100.0    # Pa → hPa
print(f'SLP DJFM: {dict(slp_djfm.sizes)}')

# Precip: raw mm/day → preprocess calculează anomalii față de REF
pr_r = climpy.preprocess(
    DATA / 'precip.nc', var='precip',
    lon_convention='[-180,180]',
    time=PERIOD, ref_period=REF,
    compute_annual=False, subtract_gm=False,
    # already_anomalies=False implicit
)
pr_djfm = climpy.seasonal_means(pr_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
pr_jjas = climpy.seasonal_means(pr_r['anom'], months=[6,7,8,9]).dropna('year', how='all')
print(f'Precip DJFM: {dict(pr_djfm.sizes)}')

## 3 — EOF SST: AMO + Tripole

EOF1 și EOF2 din SST-ul Atlantic de Nord (fără trend global) corespund uzual:
- **AMO**: pattern uniform → reflectă variabilitatea AMOC pe scale multidecadale
- **Tripole**: 3 centre nord/central/sud alternante → legat de NAO

In [ ]:
solver = climpy.EOF(annual_sst, min_variance=70)
solver.summary()
eofs = solver.eofs()
pcs  = solver.pcs()
frac = solver.variance_fraction()

In [ ]:
# ── Diagnostic EOF brut ──────────────────────────────────────────────────────
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5),
    subplot_kw={'projection': climpy.Map()})
for ax, mi, title in zip(axes, [0,1], ['EOF 1 (brut)','EOF 2 (brut)']):
    data = eofs.isel(mode=mi).values
    vm   = np.nanpercentile(np.abs(data), 98)
    dc, lc = add_cyclic_point(data, coord=eofs.lon.values)
    ax.set_extent([-80,25,-2,82], crs=climpy.Map())
    cf = ax.contourf(lc, eofs.lat.values, dc, levels=11, vmin=-vm, vmax=vm,
                     cmap=cmo.balance, transform=climpy.Map(), zorder=1)
    ax.add_feature(cfeature.LAND.with_scale('110m'), facecolor='#2E2E2E', zorder=3)
    ax.coastlines(linewidth=0.35, zorder=4)
    gl = ax.gridlines(draw_labels=True, linewidth=0.2, color='#CCCCCC', linestyle='--', zorder=5)
    gl.top_labels = gl.right_labels = False
    gl.xlabel_style = gl.ylabel_style = {'size': 6}
    plt.colorbar(cf, ax=ax, orientation='horizontal', pad=0.04, shrink=0.85,
                 label='°C').ax.tick_params(labelsize=6)
    ax.set_title(f'  {title}  ({float(frac[mi]):.1f}%)', loc='left', fontsize=8)

plt.suptitle('Diagnostic — ajustează coeficienții a, b în celula următoare',
             fontsize=8, style='italic', color='#666')
plt.tight_layout()
plt.show()

print(f'\nEOF1 N(>50N)={float(eofs.isel(mode=0).sel(lat=slice(50,80)).mean()):.3f}',
      f'  S(<30N)={float(eofs.isel(mode=0).sel(lat=slice(0,30)).mean()):.3f}')
print(f'EOF2 N(>50N)={float(eofs.isel(mode=1).sel(lat=slice(50,80)).mean()):.3f}',
      f'  S(<30N)={float(eofs.isel(mode=1).sel(lat=slice(0,30)).mean()):.3f}')

In [ ]:
# ── Combinare liniară → AMO + Tripole ───────────────────────────────────────
# Ghid: AMO = pattern UNIFORM de semn; Tripole = 3 CENTRE ALTERNANTE
# Dacă EOF1≈AMO, EOF2≈Tripole:  a_amo=1, b_amo=1  și  a_tripol=1, b_tripol=-1
# Dacă semnul e invers față de convenție (AMO warm=albastru):  a=-1

a_amo, b_amo       =  1,  1    # ← ajustează după diagnostic
a_tripol, b_tripol =  1, -1    # ← ajustează după diagnostic

eof_amo    = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=a_amo,    b=b_amo)
eof_tripol = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=a_tripol, b=b_tripol)
pc_amo     = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=a_amo,    b=b_amo)
pc_tripol  = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=a_tripol, b=b_tripol)

pc_amo_ma7    = climpy.moving_average(pc_amo,    n=7)
pc_tripol_ma7 = climpy.moving_average(pc_tripol, n=7)

def pc_to_year(pc):
    """PC cu dim 'time' (datetime fake) → DataArray cu dim 'year' (int)."""
    if 'time' in pc.dims:
        yrs = pc.time.dt.year.values
        return pc.assign_coords(year=('time', yrs)).swap_dims({'time':'year'}).drop_vars('time')
    return pc

pc_amo_yr        = pc_to_year(pc_amo)
pc_tripol_yr     = pc_to_year(pc_tripol)
pc_amo_ma7_yr    = pc_to_year(pc_amo_ma7)
pc_tripol_ma7_yr = pc_to_year(pc_tripol_ma7)

print(f'PC-AMO:    {pc_amo_yr.dims},  {int(pc_amo_yr.year[0])}–{int(pc_amo_yr.year[-1])}')
print(f'PC-Tripol: {pc_tripol_yr.dims},  {int(pc_tripol_yr.year[0])}–{int(pc_tripol_yr.year[-1])}')

## 4 — Figura EOF (Nature-style)

In [ ]:
fig = climpy.ClimPlot(
    nrows=2, ncols=2, w=climpy.NATURE_2COL, h=5.8,
    map_proj=(climpy.Map(), climpy.Map(), 'ts', 'ts'),
)
MAP_KW = dict(
    map_extent=[-80,25,-2,82], vmin=-0.5, vmax=0.5,
    cbar_step=0.25, nlevels=11, cbar_label='SST anomaly (°C)',
    cmap=CMAP_SST, show_land=True, coastlines_lw=0.35,
    lon_ticks=[-60,-30,0], lat_ticks=[0,30,60],
)
fig[0].map(eof_amo,    title=f'(a) EOF-AMO  ({float(frac[0]):.1f}%)',    **MAP_KW)
fig[1].map(eof_tripol, title=f'(b) EOF-Tripole  ({float(frac[1]):.1f}%)', **MAP_KW)
fig[2].ts(pc_amo_yr,    pc_amo_ma7_yr,    title='(c) PC-AMO',
          ylabel='Amplitude (std)', labels=['Annual','7-yr MA'], colors=['#0077BB','#CC3311'])
fig[3].ts(pc_tripol_yr, pc_tripol_ma7_yr, title='(d) PC-Tripole',
          ylabel='Amplitude (std)', labels=['Annual','7-yr MA'], colors=['#0077BB','#CC3311'])
fig.savefig('figures/fig_eof.pdf')
fig.show()

## 5 — Corelații AMO & Tripole × SAT / SLP / Precip

Indici: medie mobilă 7 ani (semnal multidecadal).  
**Stippling** = semnificativ după corecție FDR Benjamini-Hochberg (Wilks 2006).

In [ ]:
def align_yr(pc, field):
    common = sorted(set(pc['year'].values) & set(field['year'].values))
    return pc.sel(year=common), field.sel(year=common)

print('Calculez corelații + p-values (FDR) ...')

# SAT DJFM
s_a, sat_d   = align_yr(pc_amo_ma7_yr,    sat_djfm)
s_t, sat_d2  = align_yr(pc_tripol_ma7_yr, sat_djfm)
r_amo_sat,    p_amo_sat    = correlation_pvalue(s_a, sat_d,  dim='year', fdr=True)
r_tripol_sat, p_tripol_sat = correlation_pvalue(s_t, sat_d2, dim='year', fdr=True)
print('  ✓ SAT (Berkeley Earth)')

# SLP DJFM
s_a, slp_d   = align_yr(pc_amo_ma7_yr,    slp_djfm)
s_t, slp_d2  = align_yr(pc_tripol_ma7_yr, slp_djfm)
r_amo_slp,    p_amo_slp    = correlation_pvalue(s_a, slp_d,  dim='year', fdr=True)
r_tripol_slp, p_tripol_slp = correlation_pvalue(s_t, slp_d2, dim='year', fdr=True)
print('  ✓ SLP')

# Precip DJFM
s_a, pr_d    = align_yr(pc_amo_ma7_yr,    pr_djfm)
s_t, pr_d2   = align_yr(pc_tripol_ma7_yr, pr_djfm)
r_amo_pr,     p_amo_pr     = correlation_pvalue(s_a, pr_d,  dim='year', fdr=True)
r_tripol_pr,  p_tripol_pr  = correlation_pvalue(s_t, pr_d2, dim='year', fdr=True)
print('  ✓ Precip (GPCP global)')
print('Gata.')

In [ ]:
# ── Figura corelații 3×2 cu stippling ───────────────────────────────────────
fig = climpy.ClimPlot(
    nrows=3, ncols=2, w=climpy.NATURE_2COL, h=8.5,
    map_proj=(climpy.Map(-30),)*6,
)
BASE = dict(vmin=-0.7, vmax=0.7, cbar_step=0.35, nlevels=15, cbar_label='Pearson r',
            show_land=False, show_ocean=False, coastlines_lw=0.35)

panels = [
    (r_amo_sat,    p_amo_sat,    '(a) AMO × SAT DJFM (BEST)',    CMAP_SAT),
    (r_tripol_sat, p_tripol_sat, '(b) Tripole × SAT DJFM (BEST)', CMAP_SAT),
    (r_amo_slp,    p_amo_slp,    '(c) AMO × SLP DJFM',           CMAP_SLP),
    (r_tripol_slp, p_tripol_slp, '(d) Tripole × SLP DJFM',       CMAP_SLP),
    (r_amo_pr,     p_amo_pr,     '(e) AMO × Precip DJFM (GPCP)', CMAP_PRECIP),
    (r_tripol_pr,  p_tripol_pr,  '(f) Tripole × Precip DJFM (GPCP)', CMAP_PRECIP),
]
for i, (r, p, title, cmap) in enumerate(panels):
    fig[i].map(r, title=title, cmap=cmap, **BASE)
    fig[i].stipple(p.values, r['lat'].values, r['lon'].values, pthresh=0.05)
    fig.axes[i].set_facecolor('#F2F0EB')

fig.fig.text(0.5, 0.01,
    'Indici: medie mobilă 7 ani | Stippling: p<0.05 după FDR Benjamini-Hochberg (Wilks 2006)',
    ha='center', fontsize=5.5, color='#888', style='italic')
fig.savefig('figures/fig_corr.pdf')
fig.show()

## 6 — Regresii AMO & Tripole × SAT / SLP / Precip

Layout **3 rânduri × 2 coloane** (identic cu corelațiile):  
- Col. 1: AMO | Col. 2: Tripole  
- Rând 1: SAT | Rând 2: SLP | Rând 3: Precip  

Corecție autocorelare AR(1) (Quenouille 1952) + FDR.

In [ ]:
print('Calculez regresii (corecție autocorelare + FDR) ...')

# SAT DJFM
s_a, sat_d  = align_yr(pc_amo_ma7_yr,    sat_djfm)
s_t, sat_d2 = align_yr(pc_tripol_ma7_yr, sat_djfm)
sl_amo_sat,    p_sl_amo_sat    = regression_pvalue(s_a, sat_d,  dim='year', normalise=True, autocorr_correction=True, fdr=True)
sl_tripol_sat, p_sl_tripol_sat = regression_pvalue(s_t, sat_d2, dim='year', normalise=True, autocorr_correction=True, fdr=True)
print('  ✓ SAT')

# SLP DJFM
s_a, slp_d  = align_yr(pc_amo_ma7_yr,    slp_djfm)
s_t, slp_d2 = align_yr(pc_tripol_ma7_yr, slp_djfm)
sl_amo_slp,    p_sl_amo_slp    = regression_pvalue(s_a, slp_d,  dim='year', normalise=True, autocorr_correction=True, fdr=True)
sl_tripol_slp, p_sl_tripol_slp = regression_pvalue(s_t, slp_d2, dim='year', normalise=True, autocorr_correction=True, fdr=True)
print('  ✓ SLP')

# Precip DJFM
s_a, pr_d   = align_yr(pc_amo_ma7_yr,    pr_djfm)
s_t, pr_d2  = align_yr(pc_tripol_ma7_yr, pr_djfm)
sl_amo_pr,     p_sl_amo_pr     = regression_pvalue(s_a, pr_d,  dim='year', normalise=True, autocorr_correction=True, fdr=True)
sl_tripol_pr,  p_sl_tripol_pr  = regression_pvalue(s_t, pr_d2, dim='year', normalise=True, autocorr_correction=True, fdr=True)
print('  ✓ Precip')
print('Gata.')

In [ ]:
# ── Figura regresii 3×2 cu stippling ────────────────────────────────────────
fig = climpy.ClimPlot(
    nrows=3, ncols=2, w=climpy.NATURE_2COL, h=8.5,
    map_proj=(climpy.Map(-30),)*6,
)

reg_panels = [
    # (slope, pval, title, vmin, vmax, step, cbar_label, cmap)
    (sl_amo_sat,    p_sl_amo_sat,    '(a) AMO → SAT DJFM',      -0.6, 0.6, 0.3,  '°C / std(AMO)',   CMAP_SAT),
    (sl_tripol_sat, p_sl_tripol_sat, '(b) Tripole → SAT DJFM',  -0.6, 0.6, 0.3,  '°C / std(Tri)',   CMAP_SAT),
    (sl_amo_slp,    p_sl_amo_slp,    '(c) AMO → SLP DJFM',      -1.5, 1.5, 0.75, 'hPa / std(AMO)',  CMAP_SLP),
    (sl_tripol_slp, p_sl_tripol_slp, '(d) Tripole → SLP DJFM',  -1.5, 1.5, 0.75, 'hPa / std(Tri)',  CMAP_SLP),
    (sl_amo_pr,     p_sl_amo_pr,     '(e) AMO → Precip DJFM',   -0.3, 0.3, 0.15, 'mm/d / std(AMO)', CMAP_PRECIP),
    (sl_tripol_pr,  p_sl_tripol_pr,  '(f) Tripole → Precip DJFM',-0.3, 0.3, 0.15,'mm/d / std(Tri)', CMAP_PRECIP),
]

for i, (slope, pval, title, vmin, vmax, step, label, cmap) in enumerate(reg_panels):
    lat = slope['lat'].values
    lon = slope['lon'].values
    fig[i].map(slope, title=title, cmap=cmap,
               vmin=vmin, vmax=vmax, cbar_step=step, nlevels=13,
               cbar_label=label,
               show_land=False, show_ocean=False, coastlines_lw=0.35)
    fig[i].stipple(pval.values, lat, lon, pthresh=0.05)
    fig.axes[i].set_facecolor('#F2F0EB')

fig.fig.text(0.5, 0.01,
    'Corecție autocorelare AR(1) (Quenouille 1952) | '
    'Stippling: p<0.05 FDR | Indici: medie mobilă 7 ani',
    ha='center', fontsize=5.5, color='#888', style='italic')
fig.savefig('figures/fig_regr.pdf')
fig.show()

## 7 — CCA: SST × SLP Atlantic (moduri coupled AMOC-NAO)

**Barnett & Preisendorfer (1987)**: pre-filtrare EOF → SVD cross-covarianță → test semnificație prin permutare.

**Interpretare:** Mod 1 ar trebui să identifice cuplarea SST(AMO/AMOC) ↔ SLP(NAO).

In [ ]:
# SLP anual pe domeniu Atlantic-Europa pentru CCA
slp_ann_r = climpy.preprocess(
    DATA / 'slp.nc', var='slp',
    lon_convention='[-180,180]',
    time=PERIOD, ref_period=REF,
    lat=(-20, 80), lon=(-90, 40),
    compute_annual=True, subtract_gm=False,
)
annual_slp = slp_ann_r['annual'].dropna('year', how='all') / 100.0

common_yrs = sorted(set(annual_sst['year'].values) & set(annual_slp['year'].values))
sst_cca = annual_sst.sel(year=common_yrs)
slp_cca = annual_slp.sel(year=common_yrs)
print(f'SST CCA: {dict(sst_cca.sizes)}')
print(f'SLP CCA: {dict(slp_cca.sizes)}')

In [ ]:
cca = CCA(sst_cca, slp_cca, min_variance=70.0, lat_weights=True)
cca.summary()

print('\nTest semnificație CCA (500 permutări) ...')
pvals_cca = cca.significance_test(n_perm=500, rng=np.random.default_rng(42))
print(f"{'Mod':>5}  {'r_canon':>8}  {'p-value':>8}  {'Sig':>5}")
print('-' * 32)
for i, (r, p) in enumerate(zip(cca.cancorrs, pvals_cca)):
    print(f"{i+1:>5}  {r:>8.3f}  {p:>8.3f}  {'★' if p<0.05 else '·' if p<0.10 else ' ':>5}")

In [ ]:
# ── Figura CCA ───────────────────────────────────────────────────────────────
xpat  = cca.x_patterns()
ypat  = cca.y_patterns()
xts   = cca.x_timeseries()
yts   = cca.y_timeseries()
corrs = cca.canonical_correlations()
varX, varY = cca.variance_fraction()

fig = climpy.ClimPlot(
    nrows=3, ncols=2, w=climpy.NATURE_2COL, h=8.0,
    map_proj=(
        climpy.Map(), climpy.Map(),
        climpy.Map(-30), climpy.Map(-30),
        'ts', 'ts',
    ),
)

def norm_pat(da):
    v = np.nanstd(da.values)
    return da / v if v > 0 else da

# Rândul 1: SST patterns
for mi, pi in enumerate([0, 1]):
    pat = norm_pat(xpat.isel(mode=mi))
    vm  = max(0.3, float(np.nanpercentile(np.abs(pat.values), 95)))
    sig = '★' if pvals_cca[mi] < 0.05 else ''
    fig[pi].map(pat,
        title=f'({"ab"[mi]}) CCA-{mi+1} SST  r={corrs[mi]:.2f}{sig}',
        map_extent=[-80,25,-2,82],
        vmin=-vm, vmax=vm, nlevels=11, cbar_step=round(vm/2,2),
        cbar_label='Normalised amplitude',
        cmap=CMAP_SST, show_land=True, coastlines_lw=0.35,
        lon_ticks=[-60,-30,0], lat_ticks=[0,30,60],
    )

# Rândul 2: SLP patterns
for mi, pi in enumerate([2, 3]):
    pat = norm_pat(ypat.isel(mode=mi))
    vm  = max(0.5, float(np.nanpercentile(np.abs(pat.values), 95)))
    fig[pi].map(pat,
        title=f'({"cd"[mi]}) CCA-{mi+1} SLP',
        vmin=-vm, vmax=vm, nlevels=11, cbar_step=round(vm/2,2),
        cbar_label='Normalised amplitude',
        cmap=CMAP_SLP, show_land=False, show_ocean=False, coastlines_lw=0.35,
    )
    fig.axes[pi].set_facecolor('#F2F0EB')

# Rândul 3: time series
for mi, pi, lbl in [(0,4,'(e)'), (1,5,'(f)')]:
    ts_x   = xts.isel(mode=mi)
    ts_y   = yts.isel(mode=mi)
    ts_xma = climpy.moving_average(ts_x, n=7)
    fig[pi].ts(ts_x, ts_y, ts_xma,
        title=f'{lbl} CCA-{mi+1}: SST & SLP time series',
        ylabel='Canonical amplitude',
        labels=['SST canonical','SLP canonical','SST 7-yr MA'],
        colors=['#0077BB','#CC3311','#0077BB'],
        linewidths=[0.7, 0.7, 2.0],
        alphas=[0.4, 0.5, 1.0],
        shade_first=False,
    )

fig.fig.text(0.5, 0.01,
    'CCA Barnett & Preisendorfer (1987) | EOF pre-filtrare 70% | '
    'Semnificație: permutare 500 iter. | ★ p<0.05',
    ha='center', fontsize=5.5, color='#888', style='italic')
fig.savefig('figures/fig_cca.pdf')
fig.show()

## 8 — Interpretare fizică

| Mod CCA | Pattern SST | Pattern SLP | Interpretare |
|---|---|---|---|
| Mod 1 | AMO-like (uniform N. Atlantic) | NAO-like (dipol Azore–Islanda) | AMOC → feedback atmosferic |
| Mod 2 | Tripole (3 centre) | East Atlantic Pattern | Variabilitate NAO-driven |

**Convenție semn:** AMO pozitiv = N. Atlantic mai cald → SLP Azore+ / Islandă− (NAO pozitiv slăbit).  
Dacă culorile sunt inverse, setează `a_amo = -1` în celula de combinare liniară.

In [ ]:
# ── Descărcare figuri (Colab) ────────────────────────────────────────────────
try:
    from google.colab import files
    for f in ['figures/fig_eof.pdf','figures/fig_corr.pdf',
              'figures/fig_regr.pdf','figures/fig_cca.pdf']:
        files.download(f)
        print(f'  ↓ {f}')
except ImportError:
    print('Nu ești în Colab — figurile sunt în folderul figures/')